# Hands-on: churn classification on Ziggo subscriptions

In this notebook you will build several churn prediction models, each more complex and advanced than the previous.
We have facilitated in data preparation and cleaning so you don't have to.

The notebook contains several exercises. If you have any questions, just raise your hand and we'll help out shortly.


**The first exercise is in chapter 5. The first 4 give you an overview of the approach, but you don't have to do anything yourself.**

Don't spend too much time in trying to understand the fine print of the code, but try to keep the big picture. You can read the notebook later at home.

As we don't know how Google will respond to 50 sessions from the same IP-address, please be patient. ;)

**Disclaimer from BigData Republic: while you are free to copy this notebook for your own personal use, we ask that you be considerate and remove any remaining data left on your system. Thank you.**

## Table of contents


1.   Setup
2.   Loading the data
3.   Data Exploration
4.   Prepare for general ML
5.   Vanilla Pipeline
6.   Recurrent Neural Network
7.   Parameter grid search
8.   Evaluating on the test set



---
## 1 - Setup
Google Drive authentication and imports.

The cell below will prompt you to authenticate yourself to Google through a link. This is for accessing the dataset from Google Drive.
Google Colaboratory is a new and experimental service by Google. This is why it does not yet have default access to your other Google attributes.


In [0]:
!pip install -U -q PyDrive ## you will have install for every colab session

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Here we define the directory holding the dataset, this folder should contain the parquet file.
link_id = "18umCPUfyMUnypV094Q3MK6X6fX-2VrmN"
drive_file = drive.CreateFile({'id': link_id})
datafile = 'churn_meetup_dataset.parquet'
drive_file.GetContentFile(datafile)

Lastly, we install and import dependencies.

In [0]:
# Imports
# this looks weird, but we want pandas-0.23
!pip uninstall -y pandas
!pip install pandas

!pip install git+https://github.com/hyperopt/hyperopt
!pip install cython pyarrow pandas_profiling
!pip install git+https://github.com/BigDataRepublic/bdr-analytics-py.git
  
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
#%load_ext autoreload
#%autoreload 1
#%aimport bdranalytics
#import bdranalytics
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sns
from scipy.ndimage.interpolation import shift
import sklearn
from sklearn import linear_model, model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.feature_selection import RFE, RFECV
from sklearn.metrics import mean_squared_error
import itertools
from sklearn.metrics import make_scorer, r2_score
from sklearn.metrics.scorer import r2_scorer, mean_squared_error_scorer
import statsmodels
import statsmodels.tsa.api as sm
from IPython.display import display
import IPython
print("IPython version: {}".format(IPython.__version__))
print ("numpy: {}".format(np.__version__))
print ("scipy: {}".format(sc.__version__))
print ("sklearn: {}".format(sklearn.__version__))
print ("pandas: {}".format(pd.__version__))
import keras
print ("keras: {}".format(keras.__version__))
import tensorflow as tf
print ("tensorflow: {}".format(tf.__version__))
print ("Imports completed.")

### Setting up Keras
For learning neural networks, we'll be using keras. And Luckily Google Colaboratory has a simple GPU available for faster learning.

In [0]:
# Some necessary imports
import keras.backend as K
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
from keras.callbacks import ReduceLROnPlateau
from keras import losses

import datetime

Verify that the GPU is indeed available.

In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In case of non-GPU devices, we could enable multithreaded CPU to speed training up a bit

In [0]:
if False:  # Run optional if no GPU available
  import tensorflow as tf
  config = tf.ConfigProto(
      intra_op_parallelism_threads=1, 
      inter_op_parallelism_threads=1,
      allow_soft_placement=True, 
      device_count = {'CPU': 8}, 
      log_device_placement=True)
  session = tf.Session(config=config)
  K.set_session(session)

---
## 2 - Loading data

In [0]:
# Load the dataset:
Xy_orig = pd.read_parquet(datafile).sort_values(by=['customer_uid', 'month']).reset_index(drop=True)
Xy_orig.head()

### Defining the target variable
In the data set we've just loaded, the column "is_churned" contains the information we are aiming to predict.
Each row shows the information available about the user at a given timestamp and whether he has churned or not.

In practice, we don't want to know if a customer is churned now (*current* state), but if he will churn in the future (the *next* state). Therefore we shift the `is_churned` column by `-1` to get the value of 1 time ahead.

Note that this will result in a NaN value on the last index, which we therefore remove.

In [0]:
def initially_churned_rows(df):
    i=0
    while i < len(df) and df['is_churned'].iloc[i]==1:
        i+=1
    return i

def prepare_user(df):
    df = df.sort_values(by=['customer_uid', 'month'])
    df['initially_churned'] = 0
    df['initially_churned'][0:(initially_churned_rows(df))] = 1
    out_x = df.drop('is_churned', axis=1)
    out_y = df['is_churned'].shift(-1).dropna()
    out = pd.concat([
        out_x,  # the features
        out_y.to_frame() # if is churned the next month
        ], axis=1, join_axes=[df.index]).loc[out_y.index,:]
    return out

In [0]:
Xy = Xy_orig.groupby(by=['customer_uid']).apply(prepare_user)

### Selecting a hold out set

Now we want to split the data set into training, test and validation sets.
In this step, we need to take into account that multiple rows of the data set refer to the same customer (1 row per timestamp).
We want to make sure that customers are split into trainining and test set, i.e. all the rows correspoding to one given customer must be in only one of the sets. That is important in order to prevent information leakage, which could give us an overestimate of the model's performance.

We also want to make sure that ever set has the same proportion of churned and not-churned customers, so we will use a stratified split.

In [0]:
# Get all user ids:
users = Xy_orig[['customer_uid', 'is_churned']].groupby(by=['customer_uid']).max().reset_index()

# and split users into training and test sets
from sklearn.model_selection import train_test_split
users_train, users_testing, _, _ = train_test_split(users, users['is_churned'], test_size=0.33, random_state=42, stratify=users['is_churned'])
users_valid, users_test, _, _ = train_test_split(users_testing, users_testing['is_churned'], test_size=0.5, random_state=42, stratify=users_testing['is_churned'])

train_users = users_train['customer_uid'].values
validation_users = users_valid['customer_uid'].values
holdout_users = users_test['customer_uid'].values

print('{} users in the training set'.format(len(train_users)))
print('{} users in the validation set'.format(len(validation_users)))
print('{} users in the hould out/test set'.format(len(holdout_users)))

In [0]:
# Use each set of users to split the data into those same sets:

Xy = Xy.set_index(['customer_uid','month'])
Xy_train = Xy[Xy.index.get_level_values('customer_uid').isin(train_users)]
Xy_valid = Xy[Xy.index.get_level_values('customer_uid').isin(validation_users)]
Xy_test = Xy[Xy.index.get_level_values('customer_uid').isin(holdout_users)]

In [0]:
# Finally, define X (containing only features) and y (only labels):
def split_X_y(df):
    return df.drop('is_churned', axis=1), df['is_churned']

X_train, y_train = split_X_y(Xy_train)
X_valid, y_valid = split_X_y(Xy_valid)
X_test, y_test = split_X_y(Xy_test)

One of the features is called `initially_churned`, which reflects the initial churn state of the customer. That information should not be one of the features used in the model. We'll store them separately for later use.

In [0]:
train_init_churn = X_train['initially_churned']
X_train.drop(['initially_churned'], axis=1, inplace=True)

valid_init_churn = X_valid['initially_churned']
X_valid.drop(['initially_churned'], axis=1, inplace=True)

test_init_churn = X_test['initially_churned']
X_test.drop(['initially_churned'], axis=1, inplace=True)

The dataset is now split and indexed by customer id and month. For each customer we have a history of 2 years of records, corresponding to the month number ranging from 0 to 23.

## 3 - Data exploration
An essential part of the modelling task is understanding the data we are working with.
For classification, for example, is very important to check the balance between positive (churned) and negative (non-churned) samples.
Other important analysis are proportion of missing values, correlations between pairs of features and between each feature and the label etc.

In [0]:
# Number of churned and non-churned users:
print("Total nr of users:              {}".format(len(Xy.index.get_level_values('customer_uid').unique())))
print("Nr of churned users in dataset: {}".format(len(Xy.index.get_level_values('customer_uid')[Xy['is_churned']==1].unique())))

In [0]:
Xy.head(2)

In [0]:
# Missing values per feature:
def count_missing_values(col):
  print('Feature {}: {} %missing values'.format(col.name, col.isnull().sum()/len(col)*100))
  
_= Xy.apply(lambda col: count_missing_values(col), axis=0)

...

Surprise! We already cleaned the dataset for you. No missing values here.

## 4 - Prepare dataset for general ML
Most estimators cannot handle categorical features. Therefore, those features have to be first encoded using strategies such as one hot encoding, weight of evidence or leave one out.
In order to do so, we first separate all features according to their type.

Regarding categorical features, column names have been removed and so the number of unique values is essential to determine the most appropriate encoding strategy.



In [0]:
# Plot to get an idea of the data cardinality
cardinality = X_train.apply(lambda s: s.nunique()).sort_values(ascending=False)
cardinality.plot(kind='bar', figsize=(30,3))
plt.ylim((0,100))
plt.title("Number of unique values per feature")
plt.show()

All features have been either encoded, dummified or normalized. In fact, most features are categorical and have less than 20 unique values. The remaining few are numerical columns.

It is a given that all features with less than 50 unique values are categoricals.

In [0]:
categoricals = cardinality[cardinality < 50].index
numericals = cardinality[cardinality >= 50].index

print("Numericals: {} features".format(len(numericals)))
print("Categoricals: {} features".format(len(categoricals)))
print("Data types:")
X_train.dtypes.value_counts()

No categorical is higher cardinal than 40, meaning that for a first version we can probably make do with One Hot Encoding.

## 5 - Vanilla pipeline
We will start by setting up a very simple model pipeline, so that the main required steps become clear.
That means that the way each step is performed is neither ideal nor clean coded, our only goal here is to make the process understandable.

Once you understand how each step works and the purpose of it, you can move to the advanced pipeline and also implement your own solution to try to improve performance.

The vanilla pipeline will include:
* Categorical encoding using OHE only ("highly categoricals" will be ignored)
* Feature scaling
* implementation of a sklearn classifier

The vanilla pipeline does not include:
* Cross-validation + hyperparameter tunning of models 
* Encoding of  "highly categoricals" with e.g. Weight of Evidence
* Solution for class imbalance

### Encoders and scalers
First we train encoders and scalers.

In [0]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Fit encoders in the training set

# encoders
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
my_scaler = StandardScaler()

# OHE cannot deal with strings, so make sure to encode strings to numbers first.
# Luckily, we've already done that.

# Join OHE output and numerical features.
X_train_ohe = ohe.fit_transform(X_train[categoricals])
X_train_pp = np.hstack([X_train[numericals].values, X_train_ohe])

# Scale features to be standard mean and variance.
X_train_pp = my_scaler.fit_transform(X_train_pp)

print('{} cols before encoding'.format(X_train.shape[1]))
print('{} cols after encoding'.format(X_train_pp.shape[1]))

**Exercise**: also train a variance selector to remove features that don't have much information.

*Advanced*: features are time bound and user bound. Try to think of ways of checking variance in these groups.

**Answer**:

In [0]:
from sklearn.feature_selection import VarianceThreshold

# thresholder = VarianceThreshold(...)
# thresholder.fit(...)
# x_train_pp = ...

We then apply these learned encoders and scalers to the hold-out sets.

In [0]:
# Validation
X_valid_ohe = ohe.transform(X_valid[categoricals])
X_valid_pp = np.hstack([X_valid[numericals].values, X_valid_ohe])
X_valid_pp = my_scaler.transform(X_valid_pp)

# Holdout
X_test_ohe = ohe.transform(X_test[categoricals])
X_test_pp = np.hstack([X_test[numericals].values, X_test_ohe])
X_test_pp = my_scaler.transform(X_test_pp)

**Pointer**: Try to refrain from looking at the test set until the end of the notebook :-). You don't have access to the test set in real life either.

**Exercise** if you did the execise with the variance thresholder, also apply it to the validation and test set.

**Answer:**

In [0]:
# x_valid_pp = ...
# x_test_pp = ...

### Fitting a baseline

**Exercise:** train a linear regression model on the resulting train set.
Now we fit a simple linear regression model on the resulting dataset.


**Answer:**

In [0]:
from sklearn.linear_model import LogisticRegression

# lr = ...   # Logistic regression model
# y_pred_val_lr = ...  # Thresholded prediction
# y_pred_proba_val_lr = ...  # Probability prediction

### Evaluating the baseline
Evaluate our trained model on our hold-out set. Let's try the sklearn classification report and AUC metrics first.


In [0]:
from sklearn.metrics import classification_report, auc, roc_curve, confusion_matrix

def plot_auc(y, pred_proba, pred):
  
  plt.figure(figsize=(20,5))
  plt.subplot(1,2,1)
  fpr, tpr, thresholds = roc_curve(y, pred_proba[:,1])
  
  lw = 2
  plt.plot(fpr, tpr, color='darkorange',
           lw=lw, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
  plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.legend(loc="lower right")
  plt.title('ROC Curve')
  
  plt.subplot(1,2,2)
  sns.heatmap(confusion_matrix(y, pred), annot=True)
  plt.title('Confusion Matrix')
  
  plt.show()

In [0]:
print('Classification report on the validaion set:')
print(classification_report(y_valid, y_pred_val_lr))
plot_auc(y_valid, y_pred_proba_val_lr, y_pred_val_lr)

**Exercise**: why are the accuracy and AUC so high? What other metric could we have used instead?

**Answer: ** ...

---

If you don't want to think about the answer, below is an alternative evaluation plot that might provide us with more understanding.

In [0]:
from sklearn.metrics import precision_recall_curve, accuracy_score
from sklearn.metrics import average_precision_score

import matplotlib.pyplot as plt

def summary_print(y_true, y_score, dataset='Avg'):
    average_precision = average_precision_score(y_true, y_score)
    print('{0: <10} Accuracy score: {1:0.2f}'.format(dataset, accuracy_score(y_true, [round(x) for x in y_score])))
    print('{0: <10} Average precision-recall score: {1:0.2f}'.format(dataset,
          average_precision))
    
def summary_plot(y_true, y_score, dataset='Avg'):
    average_precision = average_precision_score(y_true, y_score)
    precision, recall, _ = precision_recall_curve(y_true, y_score)
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    _=plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))
    
def model_summary(y_true, y_score, dataset='Avg'):
    summary_print(y_true, y_score, dataset)
    summary_plot(y_true, y_score, dataset)

In [0]:
model_summary(y_valid, y_pred_proba_val_lr[:,1], 'Validation')

As you can see, we still have some room for improvement as our model, as the decision landscape looks strange. We also did not account for temporal patterns in the dataset, and we assumed records to be independent, even though they could potentially belong to the same customer.

**Optional Exercise:** Train and evaluate a random forest model.
What other improvements does it bring?

**Answer:**

In [0]:
# ...

## 6 - A simple recurrent neural network
We'll build a better model now, in the form of a neural network. Neural networks are an ideal model class for this hands-on as you know zero to nothing about features and domain understanding is not an option. Feature engineering is baked in neural nets.

Another viable option would be to use different types of tree based models, but those are difficult to modify to account for temporal patterns.

### Reshape dataset for neural networks
Neural networks need a different type of preprocessing. Besides normal networks, we're also dealing with temporal patterns to account for.

In order for the network to understand temporal patterns, we have to rearrange the dataset to include a time layer per customer. For time series models, the end result should be a tensor of shape  `samples * timesteps * features`.

In [0]:
num_series_train = X_train.index.get_level_values(0).nunique()
num_series_valid = X_valid.index.get_level_values(0).nunique()
num_series_test = X_test.index.get_level_values(0).nunique()  
data_dim = X_train_pp.shape[1]

# Using reshape assumes that the data has been ordered in the right way.
# In this case, the data should be and is already sorted first by customer id
# and then by measurement month.
X_train_t = X_train_pp.reshape(num_series_train, -1, data_dim)
X_valid_t = X_valid_pp.reshape(num_series_valid, -1, data_dim)
X_test_t = X_test_pp.reshape(num_series_test, -1, data_dim)

# Same treatment should be given to the labels
y_train_t = y_train.values.reshape(num_series_train, -1, 1)
y_valid_t = y_valid.values.reshape(num_series_valid, -1, 1)
y_test_t = y_test.values.reshape(num_series_test, -1, 1)

assert X_train_t.shape[0] == y_train_t.shape[0]
assert X_train_t.shape[1] == y_train_t.shape[1]

timesteps = X_train_t.shape[1]

display("Train set shape", X_train_t.shape)
display("Train label shape", y_train_t.shape)

### Optional: Coping with class imbalances
It's always important to see what class balance we actually have. Most ML algorithms minimize some average error/loss. To put more weight on the minority class, we can give them inverse weights, causing the model to minimize average class error.

We feed these weights into the model.

In [0]:
from sklearn.utils import class_weight

ratio_1 = np.sum(y_test.values) / len(y_test)
print("The ratio of 'true' labels is  : {}".format(ratio_1))
print("This means a model predicting constants will have an accuracy of {}".format(1 - ratio_1))

labels = np.unique(y_train)
class_weights = class_weight.compute_class_weight('balanced', labels, y_train)
class_weights = dict(zip(labels, class_weights))
weights_train = class_weight.compute_sample_weight(class_weights, y_train)
weights_valid = class_weight.compute_sample_weight(class_weights, y_valid)

# We have 2 years of data of each customer. If they weren't a customer for 2 years,
# they start in status 'churned' before actually being a customer.
# These datapoints don't exist when running in real life, but we need them to get fixed length timeseries
# So we correct it by setting the weight to 0
weights_train[train_init_churn == 1] = 0
weights_valid[valid_init_churn == 1] = 0

# Reshape weights for use in the neural net.
weights_train_t = weights_train.reshape(y_train_t.shape[0], -1)
weights_valid_t = weights_valid.reshape(y_valid_t.shape[0], -1)

### Defining the model architecture

As our first network, we'll use an arbitrary LSTM model.

The goal is to make you feel slightly uncomfortable. That's the best way to learn. If you're hungry for more after the MeetUp, this blog is a good place to start: [Time Series Prediction with LSTM Recurrent Neural Networks in Python with Keras ](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/)

In [0]:
def version_1():
  
  # Keras has two ways to define network structure:
  # - incrementally through the sequential API
  # - functionally, through the functional API
  model = Sequential()

  # Layer 1 - Standard dense layer.
  model.add(
        Dense(
            128,  # The output dimension of the first layer.
            # Standard activation function for neural nets. 
            # Read more here: https://en.wikipedia.org/wiki/Rectifier_(neural_networks)
            activation='relu',  
            input_shape=(X_train_t.shape[1], X_train_t.shape[2])
        )
  )
  
  # Layer 2 - Second dense layer.
  model.add(Dense(64))
  
  # Layer 3 - LSTM layers should always be preceded by a dense layer.
  # Dropout is one way of dealing with overfitting in neural networks. It effectively sets a random fraction of inputs to 0.
  # If you want to stack multiple recurrent layers, be sure to have `return_sequences` set to True.
  model.add(
      LSTM(128, return_sequences=True, activation='relu', dropout=0.2))
  
  # Layer 4 - The last recurrent sequence layer should be compressed with a TimeDistributed layer.
  # If you don't want to use an LSTM, don't use a TimeDistributed layer.
  model.add(TimeDistributed(Dense(1, activation='sigmoid')))
  
  # Selecting the optimizer for the learning procedure.
  # Feel free to tweak these values.
  optimizer = keras.optimizers.Adam(
      lr=5e-1, 
      beta_1=0.9, 
      beta_2=0.999, 
      epsilon=1e-08, 
      decay=0.0
  )
  
  # Finally compile the model with a loss function and some metrics.
  model.compile(
      loss=losses.binary_crossentropy, 
      optimizer='adam', 
      metrics=['accuracy'], 
      sample_weight_mode="temporal", 
      weighted_metrics=['accuracy']
  )
  
  model.summary()
  return model

baseline_model = version_1()

Compiling and summarising the model is a helpful wait to show your model architecture. Compare the number of parameters to the total number of training points in the data. We need to ensure our model does not overfit by configuring the network properly. The dropout only partially accounts for overfitting.

### Configure and run the network
Now to configure running our network:

In [0]:
def test(model):

    # Reduce the learning rate automatically if model performance stagnates / overfitting occurs.
    reduce_lr = ReduceLROnPlateau(
        monitor='loss', 
        factor=0.3, 
        patience=10, 
        min_lr=1e-4,
        verbose=1
    )
  
    # Stop the learning procedure before the number of epochs is reached 
    # if performance on the validation set degrades.
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        min_delta=0, 
        patience=21, 
        verbose=0, 
        mode='auto'
    )
    
    fit = model.fit(
        X_train_t, y_train_t, epochs=1000,
        validation_data=(X_valid_t, y_valid_t, weights_valid_t),
        verbose=2, 
        sample_weight=weights_train_t,  # Relative importance weighting based on class imbalance
        callbacks=[
            reduce_lr, early_stopping],
        batch_size=128
        
    )
    
    return fit

baseline_fit = test(baseline_model)

Do you understand the output of the learning algorithm?

The `reduce_lr` and `early_stopping` should collaborate well, in case the initial learning rate is way too high, it should have the possibility to be reduced enough before the early stopping kicks in.

In [0]:
print("Nr used epochs:      {}".format(len(baseline_fit.history['loss'])))
print("Train      loss: {}".format(baseline_fit.history['loss'][-1]))
print("Validation loss: {}".format(baseline_fit.history['val_loss'][-1]))

### Evaluation


In [0]:
summary_print(
    y_train_t.reshape(-1),
    baseline_fit.model.predict(X_train_t).reshape(-1),
    'Train'
)

y_valid_hat = baseline_fit.model.predict(X_valid_t).reshape(-1)
model_summary(y_valid_t.reshape(-1), y_valid_hat, 'Validation')

Is the network learning anything? Are you doing better than the previous baseline? If not, what could be going wrong? How could you improve the model?

**Exercise:** Play with the network architecture and parameter settings and see what happens.

Do you need to simplify? Add more layers? Are you overfitting? 

Try and build the best model possible.

---
## You've reached the end of the first part.

The next section covers parameter search over network architecture and challenges your skills even more.

You can skip the section and move to the test set verification.

---


## 7 - Advanced: Hyper parameter search
Go directly to section 8 if you don't want to spend time on parameter searching.

The previous network layout was quite arbitrary. We will now tune some characteristics of the layout.

This could be done manually, where we the scientist would be executing some (randomized) gradient descent.
For reproducibility we will however use the `hyperopt` package (https://github.com/hyperopt/hyperopt)


### Defining the search candidate creation

In [0]:
def create_model(depth, width, dropout=0.5, with_dense=True):
    model = Sequential()
    
    if with_dense:
        model.add(
            Dense(
                width, 
                activation='relu', 
                input_shape=(X_train_t.shape[1], X_train_t.shape[2])
            )
        )
        
    else:
        model.add(
            LSTM(
                width, 
                return_sequences=True, 
                activation='relu', 
                input_shape=(X_train_t.shape[1], X_train_t.shape[2])
            )
        )
        
    for x in range(depth-1):
        model.add(
            LSTM(
                width, 
                return_sequences=True, 
                activation='relu', 
                dropout=dropout
            )
        )
        
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    
    optimizer = keras.optimizers.Adam(
        lr=5e-1, 
        beta_1=0.9, 
        beta_2=0.999, 
        epsilon=1e-08, 
        decay=0.0
    )
    
    model.compile(
        loss=losses.binary_crossentropy, 
        optimizer='adam', 
        metrics=['accuracy']
    )
    
    model.summary()
    return model, "depth{}_width{}_dropout{}".format(depth, width, dropout)

def fit_model(model, learning_params, name):
    reduce_lr = ReduceLROnPlateau(
        monitor='loss', 
        cooldown=learning_params['lr_plateau_cooldown'], 
        factor=learning_params['lr_plateau_factor'], 
        patience=learning_params['lr_plateau_patience'], 
        min_lr=learning_params['lr_minimum'],
        verbose=1
    )
    
    label = "{}_{}".format(name, datetime.datetime.now().__str__())

    early_stopping = EarlyStopping(
        monitor='val_loss', 
        min_delta=learning_params['stop_delta'],
        patience=learning_params['stop_patience'], 
        verbose=0, 
        mode='auto'
    )
    
    fit = model.fit(
        X_train_t, y_train_t, epochs=learning_params['epochs_max'],
        validation_data=(X_valid_t, y_valid_t),
        verbose=2, 
        callbacks=[reduce_lr, early_stopping],
        batch_size= 64
    )
    
    return fit, {'label': label,
                       'val_acc': fit.history['val_acc'][-1],
                       'val_loss': fit.history['val_loss'][-1],
                       'acc': fit.history['acc'][-1],
                       'loss': fit.history['loss'][-1],
                      }

### Parameter grid
For doing a parameter search, we need to define a parameter grid.

In [0]:
learningrates1 = {
'lr_intial' : 1e-2, # higher than the default
'lr_decay' : 0.01, # for adam it is 1 / (1 + decay * t) , thus with decay 0.001 and at t==1000, lr is divided by 2
                # note that the effect of this decay is not visible in tensorboard
'lr_plateau_factor' : 0.7, # if no convergence (possibly by too high lr), we boost the lr decay
'lr_plateau_patience' : 4, # nr of consequetive epochs without improvement before we boost the lr decay
'lr_plateau_cooldown' : 10, # first do this nr of iterations at new lr before detecting plateau
'lr_minimum' : 1e-6, # the minimum lr too which we decay (for plateau detection)

'stop_patience' : 30, # if no extra improvement after this nr of steps , we terminate learning
'stop_delta' : 0.0001, # the epsilon, changes below this threshold are 'no improvement'
'epochs_max' : 1000 # limit the total nr op epochs, very high, we will stop based on plateau
}

In [0]:
# Some boiler plate for performing the search over the parameter space.
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, STATUS_FAIL, space_eval
def find_result(trials, best):
    """ 
    In a Trials object, finds the result that belongs to the the provided 'best' arguments.
    """
    array_formed = dict([(k,[v]) for k,v in best.items()])
    matched_runs = [x for x in trials.trials if x['misc']['vals']==array_formed]
    return matched_runs[0]['result']

### Running the search
Let's run the search. **Warning**: this takes a long time, so restrict the search space if you want results quickly.

In [0]:
def experiment(args):
    model, name = create_model(
            depth = int(args['depth']), 
            width=int(args['width']), 
            dropout=args['dropout'],
            with_dense=args['dense'])
    fit, results = fit_model(model, learningrates1, 'widthsearch_'+name)
    return name, fit, results
        

def objective(args):
    print(' New trial '.center(60, '='))
    print(args)
    _, _, results = experiment(args)
    return{
        'status': STATUS_OK,
        'loss':results['loss'],
        'true_loss' : results['val_loss'],
        # -- other metrics
        'acc' : results['acc'],
        'val_acc': results['val_acc'],
        # -- diagnostics
        'eval_time': time.time(),
        'label': results['label']
    }
  
  
trials = Trials()
space = {
            'type' : 'lstms',
            'width': hp.quniform('width', low=10, high=400, q=1), # note that low should be >=q to not get 0 (and therefore failed run)
            'depth' : hp.quniform('depth', low=1, high=5, q=1), # note that low should be >=q to not get 0 (and therefore failed run)
            'dropout' : hp.uniform('dropout', low=0.1, high=0.7),
            'dense' : hp.choice('with_dense', [True, False])
        }

best = fmin(
    objective,
    space=space,
    algo=tpe.suggest,
    max_evals=10,
    trials=trials)



In [0]:
print("Best choice for random variables:")
print(best) # the best choice,
print("Complete set of arguments for best run:")
best_args = space_eval(space, best)
print(best_args) # the full set of params for the best choice

Now let's get all information of the run according to the best run.  
Note that this includes the label as it can be found in TensorBoard

In [0]:
best_run = find_result(trials, best)
best_run

### Final fit
Given the hyper parameter search, we now know the best layout.

In [0]:
print("The conclusion is that run with the following arguments was best:")
print(best_args)

We'll train it again on the dataset to actually get the model instance, and subsequently verify it's performance

In [0]:
best_name, best_fit, results = experiment(best_args)

In [0]:
print("Nr used epochs:      {}".format(len(best_fit.history['loss'])))
print("Train      loss: {}".format(best_fit.history['loss'][-1]))
print("Validation loss: {}".format(best_fit.history['val_loss'][-1]))\

In [0]:
summary_print(y_train_t.reshape(-1),
             best_fit.model.predict(X_train_t).reshape(-1),
             'Train')
y_valid_true = y_valid_t.reshape(-1)
y_valid_hat = best_fit.model.predict(X_valid_t).reshape(-1)
model_summary(y_valid_true, y_valid_hat, 'Validation')

---
## 8 - Final score on the test set
When you are done tuning and selection, it is time to evaluate the performance on a hold out set.

> Here we multiply the `model_score` by -1 to get the score comparable to the previous cross validations  
> Note that the holdout test score will very likely be worse than the cv test score. One reason is that all meta params were selected to optimize that test score.

### Conclusion

In [0]:
y_test_true=y_test_t.reshape(-1)

### Quality of business rule


In [0]:
y_test_hat=np.repeat(0, len(y_test_true))
model_summary(y_test_true, y_test_hat)

### Quality of logistic regression

In [0]:
# TODO
# Do test set predictions
model_summary(y_valid, y_pred_proba_val_lr[:,1], 'Validation')

### Quality of recurrent neural network

In [0]:
y_test_hat = baseline_fit.model.predict(X_test_p, verbose=2).reshape(-1)
model_summary(y_test_true, y_test_hat)

### Quality of tuned recurrent neural network

In [0]:
y_test_hat = best_fit.model.predict(X_test_p, verbose=2).reshape(-1)
model_summary(y_test_true, y_test_hat)

You've made it to the end.

If you still want more to do, try to improve on your last model. 

## Feedback questionary
Thank you for joining our event. We would be grateful to hear about your experience:

https://www.surveymonkey.com/r/KYSLSHQ

# Further reading
If you liked this hands-on session, but would like to have a deeper understanding on certain topics, have a look at the references here.

- [Time Series Prediction with LSTM Recurrent Neural Networks in Python with Keras ](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/) - a more in depth explanation of how to build your own LSTM for time series prediction.

- [Gentle introduction to the ADAM optimization algorithm](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/) - for learning more on how to tweak parameters.

- [Algorithmic marketing for AI for marketing operations
](https://algorithmicweb.wordpress.com/) - very good overview book aimed at marketeers, covering many topics, including churn prediction.

- [Neural Network Zoo](http://www.asimovinstitute.org/neural-network-zoo/) - There's a neural network for everything. This is an overview showing the different types of networks.

- [Time to event modelling with Neural Networks](https://ragulpr.github.io/2016/12/22/WTTE-RNN-Hackless-churn-modeling/) - Survival Analysis with neural networks? This blog shows an example of how to make it work.

